In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv(r'EDAdmissionDataset.csv')

In [4]:
data.isnull().sum().sum()

44160

In [5]:
# apply the type costing
data['arrival_mode'] = data['arrival_mode'].astype('str')
data['complaint'] = data['complaint'].astype('str')   
data['diagnosis'] = data['diagnosis'].astype('str')
data['result'] = data['result'].astype('str')
data['error_code'] = data['error_code'].astype('str')

In [6]:
le = LabelEncoder()

In [7]:
data['saturation'] = data['saturation'].fillna(data['saturation'].mean())

In [8]:
data.isnull().sum().sum()

0

In [20]:
# Initialize LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to categorical columns
data['arrival_mode'] = le.fit_transform(data['arrival_mode'])
data['complaint'] = le.fit_transform(data['complaint'])
data['diagnosis'] = le.fit_transform(data['diagnosis'])
data['result'] = le.fit_transform(data['result'])
data['error_code'] = le.fit_transform(data['error_code'])

In [21]:
x = data.drop("admissions", axis=1)
y = data["admissions"]

In [22]:
x.shape

(81088, 24)

In [23]:
y.shape

(81088,)

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=72)

In [26]:
from sklearn.preprocessing import StandardScaler
# Scale the features (important for LSTM)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [27]:
# Reshape data for LSTM (samples, timesteps, features)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

# Check the shape of the data
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")


x_train shape: (56761, 1, 24)
x_test shape: (24327, 1, 24)


# **LSTM**

In [28]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Initialize the model
model = Sequential()

# Add the LSTM layers
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Add the output layer (for binary classification)
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1, 50)             15000     
                                                                 
 dropout_4 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_5 (LSTM)               (None, 1, 50)             20200     
                                                                 
 dropout_5 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_6 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [29]:
# Fit the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")


Epoch 1/10


1774/1774 [==============================] - 32s 12ms/step - loss: 0.0354 - accuracy: 0.9868 - val_loss: 3.1754e-05 - val_accuracy: 1.0000
Epoch 2/10
1774/1774 [==============================] - 17s 10ms/step - loss: 9.0719e-04 - accuracy: 0.9998 - val_loss: 2.7475e-05 - val_accuracy: 1.0000
Epoch 3/10
1774/1774 [==============================] - 15s 9ms/step - loss: 3.3213e-05 - accuracy: 1.0000 - val_loss: 3.2710e-06 - val_accuracy: 1.0000
Epoch 4/10
1774/1774 [==============================] - 16s 9ms/step - loss: 9.7223e-06 - accuracy: 1.0000 - val_loss: 1.0005e-06 - val_accuracy: 1.0000
Epoch 5/10
1774/1774 [==============================] - 15s 9ms/step - loss: 2.0708e-04 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 0.9979
Epoch 6/10
1774/1774 [==============================] - 17s 9ms/step - loss: 1.0998e-04 - accuracy: 1.0000 - val_loss: 5.0316e-07 - val_accuracy: 1.0000
Epoch 7/10
1774/1774 [==============================] - 19s 11ms/step - loss: 1.7088e-

In [30]:


# Evaluate the model on the test data
y_pred = model.predict(x_test)

# Convert probabilities to binary output (since the output layer uses sigmoid activation)
y_pred = (y_pred > 0.5)

# Generate the classification report
report = classification_report(y_test, y_pred)
print(report)


761/761 [==============================] - 5s 3ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15990
           1       1.00      1.00      1.00      8337

    accuracy                           1.00     24327
   macro avg       1.00      1.00      1.00     24327
weighted avg       1.00      1.00      1.00     24327

